# CAO Points

## Introduction

The CAO points data, available from the [CAO website](http://www.cao.ie), is published in a different format for each of the years 2019, 2020, and 2021. Each year's data, therefore, requires a different approach to acquisition, conversion to pandas DataFrame, and cleaning. The 2019 data is published in two PDF files; one for level 8 courses, and one for levels 6 and 7. The 2020 data is published as an Excel spreadsheet, and the 2021 data as preformatted text in a HTML web page.

The attributes of interest for comparison between the various years' datasets are `Course Code`, `Course Name`, `Institution Name`, `EOS`, which is the number of points achieved by the last applicant to be offered a place on the course, and `Mid`, which is the mid point between the number of points held by the highest point score and the lowest point score of the applicants offered a place on the course [1]. The 2021 data does not explicitly contain an either an `EOS` or a `Mid` column. It does provide the *Round 1* and *Round 2* points required for entry into each course as `RND1` and `RND2`. Examination of the 2020 data, which contains both an `EOS` field *and* `RND1` and `RND2` fields demonstrates that the `EOS` field is equal to the `RND2` value if it exists, otherwise the `RND1` value (```EOS = RND1 if RND1 else RND2```). As for the `Mid` field; this information does not appear to be available yet for the 2021 data.

In [33]:
# Imports
# Data analysis library
import pandas as pd
# Plotting library
import matplotlib.pyplot as plt
# PDF table parsing
from tabula import read_pdf
# Retrieval of resources from WWW
import requests
# URL construction
from requests.compat import urljoin
# Various utilities, mainly path construction
import os
# Creation of datetime strings for filenames
from datetime import datetime
# Regular expressions
import re
import numpy as np
import hashlib
from itertools import zip_longest


## Acquiring the data


In [34]:
# Location of CAO points data
base_url = 'http://www2.cao.ie/points/'
# Local data directory
data_dir = 'data/cao'
backup_dir = 'data/cao/backup'

# Dictionary of source file names mapped to the file names that will be used locally
points_urls  = ({'l8.php'                  : 'cao_2021_lvl8.html',
                 'l76.php'                 : 'cao_2021_lvl76.html',
                 'CAOPointsCharts2020.xlsx': 'cao_2020_lvl876.xlsx'
                 })

# The rest of points_urls can be assembled programmatically
# as filenames follow a pattern

# List of years as 2-digit strings from 2019 to 2005
years = [str(i).zfill(2) for i in range(19, 4, -1)]
# For each year (2019 to 2005)
for year in years:
    levels = ('lvl8', 'lvl76')

    # 2011 and 2012 data is missing second 'l' from filenames
    if year in ('12', '11'):
        levels = ('lv8', 'lv76')
        
    # For each level 
    for level in levels:
        # construct remote filename
        remote_name = level + '_' + year + '.pdf'
        # construct local filename
        local_name = 'cao_20' + year + '_' + level + '.pdf'
        # Add remote and local filenames as keys and values in points_urls dict
        points_urls[remote_name] = local_name



In [35]:

# for each of the source files 
for url in (points_urls):
    # construct url and fetch content
    response = requests.get(urljoin(base_url, url))
    
    try:
        # attempt to open any previously downloaded local file
        with open(os.path.join(data_dir, points_urls[url]), "rb") as f:
            # Calculate md5 hashes for the local file and the remote file
            md5_local = hashlib.md5(f.read()).hexdigest()
            md5_response = hashlib.md5(response.content).hexdigest()

            # Set write_flag to False if the hashes are equal and True if they are not
            write_flag = (md5_local != md5_response)
    except IOError:
        # if the local file does not exist set the write_flag to True and move on
        write_flag = True

    # If the write_flag is True
    if write_flag:
        print(f"File: {points_urls[url]} has changed since last download. Updating...")
        # split the filename into name and extension
        fname, extension = os.path.splitext(points_urls[url])
        # construct unique filename by inserting datetime string between filename and extension
        filename = fname + datetime.now().strftime("_%Y%m%d_%H%M%S") + extension

        # write the timestamped remote file to the backup directory
        with open(os.path.join(backup_dir, filename), 'wb') as f:
            f.write(response.content)

        # also write the remote file to the data directory, overwriting any previous file
        with open(os.path.join(data_dir, points_urls[url]), 'wb') as f:
            f.write(response.content)
    else:
        print(f"File: {points_urls[url]} has not changed since last download. Skipping...")


File: cao_2021_lvl8.html has not changed since last download. Skipping...
File: cao_2021_lvl76.html has not changed since last download. Skipping...
File: cao_2020_lvl876.xlsx has not changed since last download. Skipping...
File: cao_2019_lvl8.pdf has not changed since last download. Skipping...
File: cao_2019_lvl76.pdf has not changed since last download. Skipping...
File: cao_2018_lvl8.pdf has not changed since last download. Skipping...
File: cao_2018_lvl76.pdf has not changed since last download. Skipping...
File: cao_2017_lvl8.pdf has not changed since last download. Skipping...
File: cao_2017_lvl76.pdf has not changed since last download. Skipping...
File: cao_2016_lvl8.pdf has not changed since last download. Skipping...
File: cao_2016_lvl76.pdf has not changed since last download. Skipping...
File: cao_2015_lvl8.pdf has not changed since last download. Skipping...
File: cao_2015_lvl76.pdf has not changed since last download. Skipping...
File: cao_2014_lvl8.pdf has not changed 

### 2021 Points Data

In [36]:
l8 = os.path.join(data_dir, 'cao_2021_lvl8.html')
l76 = os.path.join(data_dir, 'cao_2021_lvl76.html')

# Regular expression to capture fields from lines
# Lines consist of 2 letters and 3 numbers, comprising the course code; some whitespace; 
# 50 characters which start with a non-whitespace character; some more whitespace;
# some optional non whitespace characters comprising round 1 points; some more whitespace;
# and, optionally some more non-whitespace characters comprising round 2 points if present
re_fields = re.compile('^([A-Z]{2}[0-9]{3})\s+(\S.{49})\s+(\S+)?\s+(\S+)?')

# array to hold matched groups
data = []

for datafile, level in zip((l8, l76), (8, 76)):
    # encoding=cp1252 necessary to decode some characters on page
    with open(datafile, 'r', encoding='cp1252') as f:
        for line in f:
            match = re.match(re_fields, line)
            if match:
                fields = list(match.groups())
                fields.append(level)
                data.append(fields)

# column names
columns = ['Course Code', 'Course Name', 'Rnd1', 'Rnd2', 'Level']
df = pd.DataFrame.from_records(data, columns=columns)



Create new columns to hold information currently designated by *'s and #'s in numeric columns

Create new column indicating whether the course requires a test, interview or portfolio
This is indicated by a '#' in the Rnd1 or Rnd2 column
df['Test'] = df['Rnd1'].str.contains('#', na=False) | df['Rnd2'].str.contains('#', na=False)

Create a column indicating courses where not all applicants at Rnd1 point score were offered a place
This is indicated by a '*' in the Rnd1 or Rnd2 column
df['Not All'] = df['Rnd1'].str.contains('\*', na=False) | df['Rnd2'].str.contains('\*', na=False)

Create a new column for AQA meaning All Qualified Applicants were offered a place
df['AQA'] = df['Rnd1'].str.contains('AQA', na=False) | df['Rnd2'].str.contains('AQA', na=False)

Create a new column for 'New competition for available places' which seems to be courses 
for which the points have increased in round 2. Only occurs in level 76 and is indicated 
by a 'v' in 'Rnd2' column
df['New Comp'] = df['Rnd1'].str.contains('v', na=False) | df['Rnd2'].str.contains('v', na=False)

Generate 'EOS' column. == Rnd2 if it exists, otherwise Rnd1
df['EOS'] = np.where(df['Rnd2'].isnull(), df['Rnd1'], df['Rnd2'])

Remove Non-digits from Rnd1 and Rnd2 columns and convert columns to numeric values, 
with NaNs where values are missing (errors = 'coerce')
(Because NaN is a float, the whole columns must be floats)
df['Rnd1'] = pd.to_numeric(df['Rnd1'].str.replace('[^0-9]+', '', regex=True), errors='coerce')
df['Rnd2'] = pd.to_numeric(df['Rnd2'].str.replace('[^0-9]+', '', regex=True), errors='coerce')

In [37]:
newcols = {'Test': '#', 'Not All': '\*', 'AQA': 'AQA', 'New Comp': 'v'}

for k, v in newcols.items():
    df[k] = df['Rnd1'].str.contains(v, na=False) | df['Rnd2'].str.contains(v, na=False)

# Generate 'EOS' column. == Rnd2 if it exists, otherwise Rnd1
df['EOS'] = np.where(df['Rnd2'].isnull(), df['Rnd1'], df['Rnd2'])

# Remove Non-digits from Rnd1 and Rnd2 columns and convert columns to numeric values, 
# with NaNs where values are missing (errors = 'coerce')
# (Because NaN is a float, the whole columns must be floats)
df['EOS'] = pd.to_numeric(df['EOS'].str.replace('[^0-9]+', '', regex=True), errors='coerce')

df.head(20)

,Course Code,Course Name,Rnd1,Rnd2,Level,Test,Not All,AQA,New Comp,EOS
0,AL801,Software Design for Virtual Reality and Gaming...,300,None,8,False,False,False,False,300.0
1,AL802,Software Design in Artificial Intelligence for...,313,None,8,False,False,False,False,313.0
2,AL803,Software Design for Mobile Apps and Connected ...,350,None,8,False,False,False,False,350.0
3,AL805,Computer Engineering for Network Infrastructur...,321,None,8,False,False,False,False,321.0
4,AL810,Quantity Surveying ...,328,None,8,False,False,False,False,328.0
5,AL811,Civil Engineering ...,None,None,8,False,False,False,False,NaN
6,AL820,Mechanical and Polymer Engineering ...,327,None,8,False,False,False,False,327.0
7,AL830,General Nursing ...,451*,444,8,False,True,False,False,444.0
8,AL832,Mental Health Nursing ...,440*,431,8,False,True,False,False,431.0
9,AL835,Pharmacology ...,356,None,8,False,False,False,False,356.0


### 2020 Points Data

1. Read Excel file using pandas.read_excel, specifying header row, desired columns, and row names
2. Create and populate 'Test', 'Not All', 'Matric', and 'AQA' columns
3. Remove all non-numeric characters from 'EOS' and 'Mid' and convert to numeric type

In [381]:
def tidy_cols(df):
    
    cols = ['Test', 'Not All', 'Matric', 'AQA']
    markers = ['#', '*', 'mat', 'AQA']

    for col, marker in zip(cols, markers):
        df[col] = df['EOS'].str.replace('\s', '', regex=True).str.contains(marker, na=False, regex=False)

    for col in ('EOS', 'Mid'):
        # Cast each point col to string so they can be cleaned up using string methods
        df[col] = df[col].astype(str)

        # Some pdfs (e.g. 2020, level 8) have second point values in parentheses 
        # indicating new competition for additional places which must be removed
        # or the two point values will be concatenated in the next step
        df[col] = df[col].str.replace('\(.+\)', '', regex=True)
        
        # Remove non digits and decimal points outside numbers
        df[col] = df[col].str.replace('[^0-9.]', '', regex=True).str.strip(".")

        # Cast points columns to float
        df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')
        
    return df 

In [39]:
df2020 = pd.read_excel(os.path.join(data_dir, 'cao_2020_lvl876.xlsx'), 
                       header=10, 
                       usecols="B,C,H,I,J,L", 
                       names=['Course Name', 'Course Code', 'EOS', 'EOS *', 'Mid', 'Institution Name'],
                       converters={'EOS':str,'Mid':str})

# Asterisks usually found in EOS are in a separate col in this dataset
# Move asterisks to EOS so generic parser can be used
df2020['EOS'] = np.where(df2020['EOS *'].str.contains('*', na=False, regex=False), 
    df2020['EOS'] + '*', df2020['EOS']) 
df2020 = df2020.drop('EOS *', axis=1)

df2020 = tidy_cols(df2020)
# # Create new column indicating whether the course requires a test, interview or portfolio
# # This is indicated by a '#' in the EOS column
# df2020['Test'] = df2020['EOS'].str.contains('#', na=False, regex=False)

# # Create a column indicating courses where not all applicants at EOS point score were offered a place
# # This is indicated by a '*' in the EOS column
# df2020['Not All'] = df2020['EOS'].str.contains('*', na=False, regex=False)

# # Create a column indicating courses where a matric is required
# # This is indicated by the string '+matric' in the EOS column.
# # However, the tabula table parsing has interpreted the r in matric as a cell boundary so only 'mat' 
# # remains in the EOS column and 'ic' appears in the Mid column. The 'ic' will be dealt with next 
# df2020['Matric'] = df2020['EOS'].str.contains('mat', na=False, regex=False)

# # Remove Non-digits from EOS and Mid columns and convert columns to numeric values, with NaNs where values are missing (errors = 'coerce')
# # (Because NaN is a float, the whole columns must be floats)
# # Note that EOS and Mid contain mixed dtypes and so must be converted to string before the replace operation
# df2020['EOS'] = pd.to_numeric(df2020['EOS'].str.replace('[^0-9]+', '', regex=True), errors='coerce')
# df2020['Mid'] = pd.to_numeric(df2020['Mid'].str.replace('[^0-9]+', '', regex=True), errors='coerce')

df2020.head(100)

,Course Name,Course Code,EOS,Mid,Institution Name,Test,Not All,Matric,AQA
0,International Business,AC120,209.0,280.0,American College,False,False,False,False
1,Liberal Arts,AC137,252.0,270.0,American College,False,False,False,False
2,"First Year Art & Design (Common Entry,portfolio)",AD101,NaN,NaN,National College of Art and Design,True,False,True,False
3,Graphic Design and Moving Image Design (portfo...,AD102,NaN,NaN,National College of Art and Design,True,False,True,False
4,Textile & Surface Design and Jewellery & Objec...,AD103,NaN,NaN,National College of Art and Design,True,False,True,False
...,...,...,...,...,...,...,...,...,...
95,Theatre and Performative Practices - 3 or 4 ye...,CK112,330.0,434.0,University College Cork (NUI),False,False,False,False
96,Criminology - 3 years or 4 years (Internationa...,CK113,423.0,463.0,University College Cork (NUI),False,False,False,False
97,Social Science (Youth and Community Work) - 3 ...,CK114,777.0,NaN,University College Cork (NUI),False,False,False,False
98,Social Work - Mature Applicants only,CK115,999.0,NaN,University College Cork (NUI),False,False,False,False


### 2019 Points Data

The 2019 points data is held in two PDF files, one for level 8 courses and one for levels 6 and 7.

1. Read using tabula.read_pdf()
2. If necessary remove unwanted rows and assign header row
3. Fix and rename headers
4. Fill in institution column
5. Remove rows without course codes
6. Create and populate 'Test', 'Not All', 'Matric', and 'AQA' columns
7. Remove all non-numeric characters from 'EOS' and 'Mid' and convert to numeric type


In [40]:
def read_cao_pdf(pdf_path, header_row=None, splitfirst=False, table_num=0, drop_col=None, merge_drop=None):
    
    df = read_pdf(pdf_path, pages='all', multiple_tables=False)[table_num]

    # 2016 data has a ghost column
    if drop_col is not None:
        if merge_drop is not None:
            col1 = df.columns[drop_col[0]]
            col2 = df.columns[merge_drop]
            df.loc[df[col2].isnull(), col2] = df[col1]
            
        df.drop(df.columns[drop_col], axis=1, inplace=True)
    
    df.columns = ['Course Code', 'Course Name', 'EOS', 'Mid']
    
    if header_row is not None:
        # df.columns = df.iloc[header_row]
        df.rename_axis(None, axis=1, inplace=True)
        
        # Delete rows up to header_row
        df.drop(df.index[range(0, header_row + 1)], axis=0, inplace=True)

        # Reset the index
        df.reset_index(inplace=True, drop=True)
    
    # Create a new column in the dataframe for institution name 
    # identify institution name rows as those containing null course codes
    # and add those institution names to the new institution column
    df['Institution'] = df[df['Course Code'].isnull()]['Course Name']
    #df.rename(columns={'INSTITUTION and COURSE':'Course Name'}, inplace=True)
    
    # Fill empty fields in the institution column with the most recent non-na field
    df['Institution'] = df['Institution'].fillna(method='ffill')
    
    # Remove rows containing only institution names
    df = df[df['Course Code'].notna()]
    
    # A missing vertical line causes some the pdf parser to merge rows 
    # in certain tables (e.g. 2014 levels 6 & 7)
    # If that is the case we need to shift column contents to the right 
    # then split the firat column into course code and course name
    if splitfirst:
        # Shift the values in EOS to Mid
        df['Mid'] = df['EOS']
        # Shift the values in Course Name to EOS
        df['EOS'] = df['Course Name']
        # Extract the course name from the course code column and place in Course Name column
        df['Course Name'] = df['Course Code'].str.extract('^\D\D\d{3}(.+)$')
        # Extract the course code form the Course Code column and place in COurse Code column
        df['Course Code'] = df['Course Code'].str.extract('^(\D\D\d{3})')
        
    # Remove page header rows
    df = df[df['Course Code'] != 'Course Code']
    
    df = tidy_cols(df)
    
    return df


In [41]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2019_lvl8.pdf")
df8 = read_cao_pdf(pdf_path)

# Repair LM124 Course Name
df8.loc[df8['Course Code']=='LM124', 'Course Name'] += 'ce)'

df8.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,Athlone Institute of Technology,False,False,False,False
2,AL802,Software Design with Cloud Computing,301.0,306.0,Athlone Institute of Technology,False,False,False,False
3,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,Athlone Institute of Technology,False,False,False,False
4,AL805,Network Management and Cloud Infrastructure,329.0,442.0,Athlone Institute of Technology,False,False,False,False
5,AL810,Quantity Surveying,307.0,349.0,Athlone Institute of Technology,False,False,False,False


In [42]:
# read the level 6 & 7 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2019_lvl76.pdf")
df76 = read_cao_pdf(pdf_path, header_row=7)

# Repair WD177 Course Name
df76.loc[df76['Course Code']=='WD177', 'Course Name'] += 'macy.)'

df76.head()


,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL600,Software Design,205.0,306.0,Athlone Institute of Technology,False,False,False,False
2,AL601,Computer Engineering,196.0,272.0,Athlone Institute of Technology,False,False,False,False
3,AL602,Mechanical Engineering,258.0,424.0,Athlone Institute of Technology,False,False,False,False
4,AL604,Civil Engineering,252.0,360.0,Athlone Institute of Technology,False,False,False,False
5,AL630,Pharmacy Technician,306.0,366.0,Athlone Institute of Technology,False,False,False,False


#### Merge dataframes

In [43]:
# add level 8 column to both dataframes
df8['Level 8'] = True
df76['Level 8'] = False

In [44]:
# conatenate level with levels 6 & 7
df = pd.concat([df8, df76], ignore_index=True)

# Rename column names to include year   
df = df.rename({'Course Name': 'Course Name 2019', 
                'EOS': 'EOS 2019', 
                'Mid':'Mid 2019', 
                'Test':'Test 2019', 
                'Not All': 'NotAll 2019',
                'Matric': 'Matric 2019',
                'AQA': 'AQA 2019',
                'Level 8': 'Level8 2019'}, axis=1)

In [45]:
df.tail()

,Course Code,Course Name 2019,EOS 2019,Mid 2019,Institution,Test 2019,NotAll 2019,Matric 2019,AQA 2019,Level8 2019
1386,WD188,Applied Health Care,206.0,339.0,Waterford Institute of Technology,False,False,False,False,False
1387,WD205,Molecular Biology with Biopharmaceutical Science,208.0,441.0,Waterford Institute of Technology,False,False,False,False,False
1388,WD206,Electronic Engineering,191.0,322.0,Waterford Institute of Technology,False,False,False,False,False
1389,WD207,Mechanical Engineering,179.0,330.0,Waterford Institute of Technology,False,False,False,False,False
1390,WD208,Manufacturing Engineering,195.0,330.0,Waterford Institute of Technology,False,False,False,False,False


In [46]:
# export dataframe to csv
df.to_csv('data/cao/cao_points_2019.csv')

### 2018

In [47]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2018_lvl8.pdf")
df8_18 = read_cao_pdf(pdf_path, header_row=7)

df8_18.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL801,Software Design (Game Development or Cloud Com...,295.0,326.0,Athlone Institute of Technology,False,False,False,False
2,AL810,Quantity Surveying,300.0,340.0,Athlone Institute of Technology,False,False,False,False
3,AL820,Mechanical and Polymer Engineering,299.0,371.0,Athlone Institute of Technology,False,False,False,False
4,AL830,General Nursing,418.0,440.0,Athlone Institute of Technology,False,False,False,False
5,AL832,Psychiatric Nursing,377.0,388.0,Athlone Institute of Technology,False,False,False,False


In [48]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2018_lvl76.pdf")
df76_18 = read_cao_pdf(pdf_path, header_row=7)

df76_18.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL601,Electronics and Computer Engineering,240.0,321.0,Athlone Institute of Technology,False,False,False,False
2,AL602,Mechanical Engineering,201.0,299.0,Athlone Institute of Technology,False,False,False,False
3,AL604,Civil Engineering,243.0,320.0,Athlone Institute of Technology,False,False,False,False
4,AL630,Pharmacy Technician,306.0,388.0,Athlone Institute of Technology,False,False,False,False
5,AL631,Dental Nursing,307.0,348.0,Athlone Institute of Technology,False,False,False,False


### 2017

In [49]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2017_lvl8.pdf")
df8_17 = read_cao_pdf(pdf_path)

df8_17.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL801,Software Design (Game Development or Cloud Com...,290.0,329.0,Athlone Institute of Technology,False,False,False,False
2,AL810,Quantity Surveying,311.0,357.0,Athlone Institute of Technology,False,False,False,False
3,AL820,Mechanical and Polymer Engineering,300.0,336.0,Athlone Institute of Technology,False,False,False,False
4,AL830,General Nursing,398.0,418.0,Athlone Institute of Technology,False,True,False,False
5,AL832,Psychiatric Nursing,378.0,389.0,Athlone Institute of Technology,False,False,False,False


In [50]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2017_lvl76.pdf")
df76_17 = read_cao_pdf(pdf_path)

df76_17.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL601,Electronics and Computer Engineering,228.0,420.0,Athlone Institute of Technology,False,False,False,False
2,AL602,Mechanical Engineering,212.0,303.0,Athlone Institute of Technology,False,False,False,False
3,AL604,Civil Engineering,NaN,281.0,Athlone Institute of Technology,False,False,False,True
4,AL630,Pharmacy Technician,290.0,356.0,Athlone Institute of Technology,False,False,False,False
5,AL631,Dental Nursing,273.0,336.0,Athlone Institute of Technology,False,False,False,False


### 2016

In [51]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2016_lvl8.pdf")
df8_16 = read_cao_pdf(pdf_path, header_row=6, drop_col=[4])

df8_16.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL801,Software Design (Game Development or Cloud Com...,300.0,340.0,Athlone Institute of Technology,False,False,False,False
2,AL810,Quantity Surveying,315.0,355.0,Athlone Institute of Technology,False,False,False,False
3,AL820,Mechanical and Polymer Engineering,295.0,340.0,Athlone Institute of Technology,False,False,False,False
4,AL830,General Nursing,425.0,440.0,Athlone Institute of Technology,False,True,False,False
5,AL831,Mature Applicants General Nursing,181.0,185.0,Athlone Institute of Technology,True,False,False,False


In [52]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2016_lvl76.pdf")
df76_16 = read_cao_pdf(pdf_path, header_row=6, drop_col=[4])

df76_16.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL601,Electronics and Computer Engineering,205.0,295.0,Athlone Institute of Technology,False,False,False,False
2,AL602,Mechanical Engineering,205.0,305.0,Athlone Institute of Technology,False,False,False,False
3,AL604,Civil Engineering,280.0,370.0,Athlone Institute of Technology,False,False,False,False
4,AL630,Pharmacy Technician,270.0,383.0,Athlone Institute of Technology,False,False,False,False
5,AL631,Dental Nursing,275.0,365.0,Athlone Institute of Technology,False,False,False,False


### 2015

In [53]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2015_lvl8.pdf")
df8_15 = read_cao_pdf(pdf_path, header_row=14)

df8_15.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL801,Software Design (Game Development or Cloud Com...,280.0,345.0,Athlone Institute of Technology,False,False,False,False
2,AL820,Mechanical and Polymer Engineering,315.0,355.0,Athlone Institute of Technology,False,False,False,False
3,AL830,General Nursing,420.0,435.0,Athlone Institute of Technology,False,False,False,False
4,AL831,Mature Applicants General Nursing,176.0,182.0,Athlone Institute of Technology,True,True,False,False
5,AL832,Psychiatric Nursing,390.0,400.0,Athlone Institute of Technology,False,False,False,False


In [54]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2015_lvl76.pdf")
df76_15 = read_cao_pdf(pdf_path, header_row=13)

df76_15.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL601,Electronics and Computer Engineering,210.0,315.0,Athlone Institute of Technology,False,False,False,False
2,AL602,Mechanical Engineering,175.0,260.0,Athlone Institute of Technology,False,False,False,False
3,AL604,Civil Engineering,175.0,305.0,Athlone Institute of Technology,False,False,False,False
4,AL630,Pharmacy Technician,270.0,390.0,Athlone Institute of Technology,False,False,False,False
5,AL631,Dental Nursing,265.0,330.0,Athlone Institute of Technology,False,False,False,False


### 2014

In [55]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2014_lvl8.pdf")
df8_14 = read_cao_pdf(pdf_path, header_row=13)

df8_14.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL801,Software Design (Common Entry,280.0,335.0,ATHLONE IT,False,False,False,False
2,AL820,Mechanical and Polymer Engineering,315.0,365.0,ATHLONE IT,False,False,False,False
3,AL830,General Nursing,410.0,420.0,ATHLONE IT,False,False,False,False
4,AL831,Mature Applicants General Nursing,169.0,173.0,ATHLONE IT,True,False,False,False
5,AL832,Psychiatric Nursing,390.0,395.0,ATHLONE IT,False,False,False,False


In [56]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2014_lvl76.pdf")
df76_14 = read_cao_pdf(pdf_path, header_row=12, splitfirst=True)

df76_14.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL601,Electronics and Computer Engineering,185.0,290.0,ATHLONE IT,False,False,False,False
2,AL602,Mechanical Engineering,180.0,255.0,ATHLONE IT,False,False,False,False
3,AL604,Civil Engineering,95.0,250.0,ATHLONE IT,False,False,False,False
4,AL630,Pharmacy Technician,320.0,390.0,ATHLONE IT,False,False,False,False
5,AL631,Dental Nursing,265.0,335.0,ATHLONE IT,False,False,False,False


### 2013

In [57]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2013_lvl8.pdf")
df8_13 = read_cao_pdf(pdf_path, header_row=10)

df8_13.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
2,AL802,Software Design (Games Development),275.0,325.0,ATHLONE IT,False,False,False,False
3,AL803,Software Design (Cloud Computing),280.0,345.0,ATHLONE IT,False,False,False,False
4,AL830,General Nursing,410.0,415.0,ATHLONE IT,False,True,False,False
5,AL831,Mature Applicants General Nursing,566.0,581.0,ATHLONE IT,True,False,False,False
6,AL832,Psychiatric Nursing,395.0,400.0,ATHLONE IT,False,False,False,False


In [58]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2013_lvl76.pdf")
df76_13 = read_cao_pdf(pdf_path, header_row=10)

df76_13.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
3,AL601,Electronics and Computer Engineering,205.0,285.0,ATHLONE IT,False,False,False,False
4,AL604,Civil Engineering,165.0,260.0,ATHLONE IT,False,False,False,False
5,AL630,Pharmacy Technician,305.0,400.0,ATHLONE IT,False,False,False,False
6,AL631,Dental Nursing,300.0,350.0,ATHLONE IT,False,False,False,False
7,AL632,Science (Bioscience/Chemistry),160.0,300.0,ATHLONE IT,False,False,False,False


### 2012

In [59]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2012_lv8.pdf")
df8_12 = read_cao_pdf(pdf_path, header_row=11)

df8_12.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
3,AL802,Software Design (Games Development),300.0,340.0,ATHLONE IT,False,False,False,False
4,AL803,Software Design (Web Development),310.0,335.0,ATHLONE IT,False,False,False,False
5,AL805,Construction Technology and Management,NaN,NaN,ATHLONE IT,False,False,False,False
6,AL830,General Nursing,415.0,430.0,ATHLONE IT,False,True,False,False
7,AL831,Mature Applicants General Nursing,233.0,235.0,ATHLONE IT,True,False,False,False


In [60]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2012_lv76.pdf")
df76_12 = read_cao_pdf(pdf_path, header_row=10)

df76_12.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
3,AL601,Electronics and Computer Engineering,200.0,325.0,ATHLONE IT,False,False,False,False
4,AL602,Mechanical Engineering,200.0,285.0,ATHLONE IT,False,False,False,False
5,AL603,Construction Studies,195.0,280.0,ATHLONE IT,False,False,False,False
6,AL604,Civil Engineering,240.0,280.0,ATHLONE IT,False,False,False,False
7,AL630,Pharmacy Technician,275.0,365.0,ATHLONE IT,False,False,False,False


### 2011

In [61]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2011_lv8.pdf")
df8_11 = read_cao_pdf(pdf_path, header_row=24)

df8_11.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
0,AL032,Software Design (Games Development),285.0,330.0,NaN,False,False,False,False
1,AL033,Toxicology,240.0,330.0,NaN,False,False,False,False
2,AL034,Software Design (Web Development),285.0,340.0,NaN,False,False,False,False
3,AL035,Construction Technology and Management,265.0,315.0,NaN,False,False,False,False
4,AL050,Business,270.0,325.0,NaN,False,False,False,False


In [62]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2011_lv76.pdf")
df76_11 = read_cao_pdf(pdf_path, header_row=19)

df76_11.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL001,Business,160.0,280.0,ATHLONE IT,False,False,False,False
2,AL002,Culinary Arts,155.0,215.0,ATHLONE IT,False,False,False,False
3,AL003,Office Management,NaN,190.0,ATHLONE IT,False,False,False,True
4,AL004,Bar Supervision,135.0,185.0,ATHLONE IT,False,False,False,False
5,AL006,Applied Social Studies in Social Care,315.0,345.0,ATHLONE IT,False,False,False,False


### 2010

In [63]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2010_lvl8.pdf")
df8_10 = read_cao_pdf(pdf_path, header_row=17)

df8_10.tail()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
936,Philosophy:,480 - 570,NaN,NaN,WATERFORD INSTITUTE OF TECHNOLOGY,False,False,False,False
937,Psychology:,570,NaN,NaN,WATERFORD INSTITUTE OF TECHNOLOGY,False,False,False,False
938,Russian:,425 - 535*,NaN,NaN,WATERFORD INSTITUTE OF TECHNOLOGY,False,False,False,False
939,Sociology:,450* - 570,NaN,NaN,WATERFORD INSTITUTE OF TECHNOLOGY,False,False,False,False
940,Spanish:,450* - 535*,NaN,NaN,WATERFORD INSTITUTE OF TECHNOLOGY,False,False,False,False


In [64]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2010_lvl76.pdf")
df76_10 = read_cao_pdf(pdf_path, header_row=None, table_num=1, drop_col=[1], merge_drop=2)

df76_10.head()

CSVParseError: Error failed to create DataFrame with different column tables.
Try to set `multiple_tables=True`or set `names` option for `pandas_options`. 
, caused by ParserError('Error tokenizing data. C error: Expected 2 fields in line 12, saw 5\n')

### 2009

In [ ]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2009_lvl8.pdf")
df8_09 = read_cao_pdf(pdf_path, header_row=17)

df8_09.head()

NameError: name 'read_cao_pdf' is not defined

In [ ]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2009_lvl76.pdf")
df76_09 = read_cao_pdf(pdf_path, header_row=11)

df76_09.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL001,Business,125.0,260.0,ATHLONE IT,False,False,False,False
2,AL003,Office Management,120.0,205.0,ATHLONE IT,False,False,False,False
3,AL005,Front Office Management,NaN,210.0,ATHLONE IT,False,False,False,True
4,AL006,Applied Social Studies in Social Care,340.0,380.0,ATHLONE IT,False,False,False,False
5,AL007,Childcare Supervisory Management,300.0,330.0,ATHLONE IT,False,False,False,False


### 2008

In [ ]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2008_lvl8.pdf")
df8_08 = read_cao_pdf(pdf_path, header_row=27)

df8_08.tail()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
818,WD816,Mature Applicants General Nursing,174.0,184.0,WATERFORD INSTITUTE OF TECHNOLOGY,True,False,False,False
819,WD817,Mature Applicants Psychiatric Nursing,160.0,165.0,WATERFORD INSTITUTE OF TECHNOLOGY,True,False,False,False
820,WD820,Mature Applicants Intellectual Disability Nursing,157.0,160.0,WATERFORD INSTITUTE OF TECHNOLOGY,True,False,False,False
825,Two Subject Moderatorship Course ...,NaN,NaN,NaN,WATERFORD INSTITUTE OF TECHNOLOGY,False,False,False,False
851,details are available at http://www.tcd.ie/Adm...,NaN,NaN,NaN,Spanish,False,False,False,False


In [ ]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2008_lvl76.pdf")
df76_08 = read_cao_pdf(pdf_path, header_row=24)

df76_08.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL001,Business,NaN,285.0,ATHLONE IT,False,False,False,True
2,AL003,Office Management,NaN,235.0,ATHLONE IT,False,False,False,True
3,AL005,Front Office Management,150.0,280.0,ATHLONE IT,False,False,False,False
4,AL006,Applied Social Studies in Social Care,300.0,355.0,ATHLONE IT,False,False,False,False
5,AL007,Childcare Supervisory Management,280.0,325.0,ATHLONE IT,False,False,False,False


### 2007

In [395]:
def get_cao_broadsheet(pdf_path, top, height, width, col_locs, runover, header_row=0):

    # List to hold dataframes
    tables = []
    for i, col_loc in enumerate(col_locs):
        # table area in this page column
        area = [top, col_loc, top + height, col_loc + width]
        # tables will be a list containing three lists, one holding all of the left page column tables, 
        # one all the centre column tables, and one all of the right column tables
        tables.append(read_pdf(pdf_path, pages="all", multiple_tables=True, area=area, pandas_options={'header': None}))

    # All of those above can be shifted to the left
    # Iterate through lists of lists of dataframes
    for df_list in tables:
        # Iterate through all dataframes in list
        for df in df_list:
            # If the dataframe has more than four columns
            if len(df.columns) > 4:
                # the last column is not wanted
                extra_col = df.iloc[:,-1]
                # if the number of rows in the dataframe 
                # is less than the number of na values in 
                # the extra row then there must be some data 
                # in the extra column that needs to be moved 
                # before the column is dropped
                if df.shape[0] > extra_col.isna().sum():
                    # Find the rows which hold data in the extra column 
                    # and shift all values one cell to the left
                    df[extra_col.notna()] = df[extra_col.notna()].shift(periods=-1, axis=1)
                
                # drop the extra column
                df.drop(df.columns[4], axis=1, inplace=True)

    # Transpose table list so that each sublist represents a page
    # and each dataframe represents a column in that page
    pages = [list(table) for table in zip_longest(*tables)]

    #Iterate over lists representing pages, starting with page 2 as 
    # page one has no previous page to push rows up to
    for page in range(1, len(pages)):
        # Get the number of rows which have run on from the previous page
        num_rows = runover[page]
        # iterate through dataframes representing page columns
        for i, col in enumerate(pages[page]):
            if col is not None:
                # copy the runover rows
                rows = col.head(num_rows)
                # append the runover rows to the dataframes representing the previous page's columns
                pages[page - 1][i] = pages[page - 1][i].append(rows, ignore_index=True)
                # drop the runover rows from the dataframes they had run over into
                col.drop(rows.index, inplace=True)

    # Flatten the list so that all data frames are in the 
    # correct order for concatenation
    table_cols = [col for page in pages for col in page]

    # The last two elements are None so remove them
    del(table_cols[-2:])

    # concatenate all of the column tables into a single dataframe
    df = pd.concat(table_cols)
    # Set column names
    df.columns = ['Course Code', 'Course Name', 'EOS', 'Mid']
    # reset the index
    df.reset_index(drop=True, inplace=True)

    # Remove all rows up to the header row which contain no data
    df.drop(df.index[0:header_row], inplace=True)

    # Remove rows where all values are NaN
    df.drop(df.index[pd.isnull(df).all(1)], inplace=True)

    # Rows in which Course Code is NaN and Course Name is not all caps, 
    # non-alphanumeric characters, and the words of, the, and and 
    # do not contain usable data
    df.drop(df[~(df['Course Name'].str.contains(
        '^[A-Z\s\Woftheand]+$', na=False)) & df['Course Code'].isna()], axis=1).index

    # Create a new column in the dataframe for institution name 
    # identify institution name rows as those containing null course codes
    # and add those institution names to the new institution column
    df['Institution'] = df[df['Course Code'].isnull() | df['Course Code'].str.contains('Code')]['Course Name']

    # Some Institution rows have the word 'Code' in the Course Code column (in 2006 pdfs)
    

    # Fill empty fields in the institution column with the most recent non-na field
    df['Institution'] = df['Institution'].fillna(method='ffill')

    # Remove rows containing only institution names
    df = df[df['Course Code'].notna()]

    # add remaining columns and clean up
    df = tidy_cols(df)

    # reset the index
    df.reset_index(drop=True, inplace=True)

    # Search in course code column reveals some bad rows
    # Drop any rows where Course Code does not follow /d/d/D/D/D pattern
    df.drop(df[~df['Course Code'].str.contains('^\D\D\d\d\d$', regex=True)].index, inplace=True)

    return df



In [376]:
pdf_path = os.path.join(data_dir, "cao_2007_lvl8.pdf")

# distance in points of top of table from top of page, 
# height of table, and width of table
top, height, width  = (18.875, 568, 246)

# distance in points of left edge of page column from left edge of page
col_locs = (18.375, 260.625, 509.625)

# Table columns run over to next page in most cases
# The 'runover' variable holds the number of rows in each page
# that need to be push back up to the previous page
runover = [0, 2, 4, 6, 7, 9, 0]

df8_07 = get_cao_broadsheet(pdf_path, top=top, height=height, width=width, col_locs=col_locs, runover=runover, header_row=18)

/tmp/ipykernel_40570/2513609360.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df['EOS'].str.replace('\s', '').str.contains(marker, na=False, regex=False)


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(df8_07)

In [374]:
pdf_path = os.path.join(data_dir, "cao_2007_lvl76.pdf")

# distance in points of top of table from top of page, 
# height of table, and width of table
top, height, width  = (18.875, 568, 246)

# distance in points of left edge of page column from left edge of page
col_locs = (18.375, 260.625, 509.625)

# Table columns run over to next page in most cases
# The 'runover' variable holds the number of rows in each page
# that need to be push back up to the previous page
runover = [0, 2, 4, 0, 0]

df76_07 = get_cao_broadsheet(pdf_path, top=top, height=height, width=width, col_locs=col_locs, runover=runover,header_row=15)

/tmp/ipykernel_40570/2513609360.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df['EOS'].str.replace('\s', '').str.contains(marker, na=False, regex=False)


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(df76_07)

### 2006

In [396]:
pdf_path = os.path.join(data_dir, "cao_2006_lvl8.pdf")
top, height, width = (18.125, 556.5, 324.75)
col_locs = (19.125, 353.625)
runover = [0, 0, 4, 7, 7, 12, 15, 11, 20]
header_row = 20

df8_06 = get_cao_broadsheet(pdf_path=pdf_path, top=top, height=height, 
                            width=width, col_locs=col_locs, runover=runover, 
                            header_row=header_row)

In [397]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(df8_06)

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
0,AL031,Computer and Software Engineering,295.0,345.0,ATHLONE IT,False,False,False,False
1,AL033,Toxicology,255.0,305.0,ATHLONE IT,False,False,False,False
2,AL035,Construction Technology and Management,300.0,340.0,ATHLONE IT,False,False,False,False
3,AL050,Business,275.0,320.0,ATHLONE IT,False,False,False,False
4,AL051,Accounting,270.0,345.0,ATHLONE IT,False,False,False,False
5,AL052,Design (Visual Communications),675.0,900.0,ATHLONE IT,True,False,False,False
6,AL053,Pharmaceutical Sciences,NaN,NaN,ATHLONE IT,False,False,False,False
7,AL054,Social Care Practice,320.0,345.0,ATHLONE IT,False,False,False,False
8,AL055,Languages & Business Administration,NaN,NaN,ATHLONE IT,False,False,False,False
9,AL110,General Nursing (Standard Code),365.0,390.0,ATHLONE IT,False,True,False,False


### 2005 

In [ ]:
# read the level 8 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2005_lvl8.pdf")
df8_05 = read_cao_pdf(pdf_path, header_row=10)

df8_05.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL031,Computer and Software Engineering,275.0,315.0,ATHLONE IT,False,False,False,False
2,AL033,Toxicology,255.0,330.0,ATHLONE IT,False,False,False,False
3,AL035,Construction Technology and Management,290.0,345.0,ATHLONE IT,False,False,False,False
4,AL050,Business Studies,300.0,340.0,ATHLONE IT,False,False,False,False
5,AL051,Accountancy,300.0,340.0,ATHLONE IT,False,False,False,False


In [ ]:
# read the level 76 pdf, extracting tables into a single dataframe
pdf_path = os.path.join(data_dir, "cao_2005_lvl76.pdf")
df76_05 = read_cao_pdf(pdf_path, header_row=9)

df76_05.head()

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
1,AL001,Business Studies,180.0,305.0,ATHLONE IT,False,False,False,False
2,AL003,Office Information Systems,135.0,250.0,ATHLONE IT,False,False,False,False
3,AL005,Front Office Management,120.0,220.0,ATHLONE IT,False,False,False,False
4,AL006,Applied Social Studies in Social Care,305.0,345.0,ATHLONE IT,False,False,False,False
5,AL009,Plastics Engineering,175.0,295.0,ATHLONE IT,False,False,False,False


## Analysing the data

## Conclusion

## References

[1] https://www.independent.ie/life/family/learning/understanding-your-cao-course-guide-26505318.html


https://tabula-py.readthedocs.io/en/latest/faq.html#how-to-use-area-option